<a href="https://colab.research.google.com/github/dkraynev/Pandas/blob/main/001_Dashboard_local.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install flask_caching

In [2]:
!pip install flask

In [3]:
!pip install dash

In [4]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from datetime import date, datetime, timedelta
from dateutil.relativedelta import relativedelta
import requests
import io
from io import StringIO
from flask_caching import Cache
from flask import Flask, jsonify, request
import dash
import dash_bootstrap_components as dbc
from dash import dcc, html, Dash
from dash_bootstrap_templates import load_figure_template
from dash.dependencies import Output, Input, State
import traceback

In [ ]:
# decorator
def measure_time(label=None):
    def decorator(func):
        def wrapper(*args, **kwargs):
            start = time.perf_counter()
            result = func(*args, **kwargs)
            end = tiem.perf_counter()
            elapsed = end - start
            name = label if label else func.__name__
            print(f"[DEBUG] Функция'{name}' заняла {elapsed:.2f} секунд")
            return result
        return wrapper
    return decorator

In [ ]:
GEO={'US': 'USA', 'CA': 'Canada', 'ME': 'Mexico'}

year_begin_date = datetime(2025, 1, 1)
filtering_date= datetime(2025, 2, 10)

In [ ]:
tracking = pd.read_csv(r'C:\Datasets\tracking.csv', delimiter=',', low_memory=False)

In [ ]:
def load_ngr_date(file_path, start_date, end_date, brand_name=None):
    df = pd.DataFrame()
    start_date = pd.to_date_time(start_date)
    end_date = pd.to_datetime(end_date)
    current_date = start_date

    while current_date <= end_date:
        if brand_name:
            try:
                df_new = pd.read_csv(filepath.format(brand_name, current_date.strftime('%Y-%m')), low_memory=False)
                df = pd.concat([df, df_new])
                del df_new
            except Exception as e:
                    print(f'Faled to load data for {brand_name} at {current_date.strftime('%Y-%m')}: {e}')
        else:
            for brand in GEO.values():
                try:
                    df_new = pd.read_csv(file_path.format(brand, current_date.strftime('%Y-%m')), low_memory=False)
                    df = pd.concat([df, df_new])
                    del df_new
                except Exception as e:
                    print(f'Failed to load for {brand} at {current_date.strftime('%Y-%m')}: {e}')
    return df

In [ ]:
def load_revshare_data(file_path, start_date, end_date, brand_name=None):
    df = pd.DateFrame()
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_date_time(end_date)

    if brand_name:
        try:
            df_new = pd.read_csv(file_path.format(brand), low_memory=False)
            df = pd.concat([df, df_new])
            del df_new
        except Exception as e:
            print(f'Failed to load data for {brand_name}: {e}')
    else:
        for brand in GEO.values():
            try:
                df_new = pd.read_csv(file_path.format(brand), low_memory=False)
                df = pd.concat([df, df_new])
                del df_new
            except Exception as e:
                print(f'Failed to load data for {brand}: {e}')

    df = date_func(df)

    # Exsure 'RegTrucn' is in datetime format for filtering
    if 'RegTrunc' in df.columns:
        df['RegTrunc'] = pd.to_datetime(df['RegTrunc'])
        df = df[(df['RegTrunc'] >= start_date) & (df['RegTrunc'] <= end_date)]
    else:
        print("Comumn 'RegTrunc' not found in the DataFrame")

    return df